In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:

print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:

full_data_rows_list = [] 
    

for f in file_path_list:


    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

print(len(full_data_rows_list))

#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:

# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(["127.0.0.1"])

session = cluster.connect()

#### Create Keyspace

In [6]:

try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:

session.set_keyspace("sparkify")

In [8]:

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_info_by_session (
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            length FLOAT,
            PRIMARY KEY (sessionId, itemInSession));
    """)
except Exception as e:
    print(e)

                    

In [9]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Query 1

 - Grabs the artist, song and length columns from the table
 - For item 4 in sessionId 338
 - Partition key has to be the sessionId as this is the natural split for the data for this query - composite key (itemInSession) is required      to ensure the primary key is unique

In [10]:

query = session.execute("""
        SELECT artist, song, length
        FROM song_info_by_session
        WHERE
            sessionId=338 AND
            itemInSession=4;
""")
query.current_rows

[Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)]

In [11]:

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS artist_song_user_by_session (
            userId INT,
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            PRIMARY KEY ((userId, sessionId), itemInSession));
    """)
except Exception as e:
    print(e)        

In [12]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user_by_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

## Query 2

 - Gets the name of the artist, song, user's first name and user's last name
 - For a particular user and a particular session
 - It makes sense to use session and user id as the partition key to fit the query
 - However this is not enough to ensure uniqueness of Primary Key - therefore we use itemInSession as Clustering Column

In [13]:
query = session.execute("""
        SELECT artist, song, firstName, lastName
        FROM artist_song_user_by_session
        WHERE
            userId=10 AND sessionId=182
        ORDER BY itemInSession;
        """)

query.current_rows

[Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz'),
 Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz'),
 Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz'),
 Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')]

In [14]:

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_by_song (
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            userId INT,
            PRIMARY KEY (song, userId));
    """)
except Exception as e:
    print(e)           

In [15]:
file = "event_datafile_new.csv"

with open(file, encoding = "utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO user_by_song (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

## Query 3:

 - This query retrieves the names of people who listened to the song "All Hands Against His Own"
 - It selects the firstName and lastName columns from the table
 - It applies the condition that the song column must have the value 'All Hands Against His Own"
 - The partition key for this query was song - a natural fit for our query which is concerned with a specific song
 - The clustering column userId is sufficient by itself to make the primary key unique
 

In [32]:
query = session.execute("""
        SELECT firstName, lastName
        FROM user_by_song
        WHERE song='All Hands Against His Own';
""")

query.current_rows

[Row(firstname='Jacqueline', lastname='Lynch'),
 Row(firstname='Sara', lastname='Johnson'),
 Row(firstname='Tegan', lastname='Levine')]

In [17]:
try:
    session.execute("DROP TABLE artist_song_user_by_session")
    session.execute("DROP TABLE song_info_by_session")
    session.execute("DROP TABLE user_by_song")
except Exception as e:
    print(e)
    

In [18]:
session.shutdown()
cluster.shutdown()